# Code Written by:
**Shweta Tiwari**
*20 Oct 2023*


## Algorithm: FIRST FOLLOW

In [1]:
import time

# Algorithm

In [2]:
%%time
def first_and_follow(grammar):
    # first & follow sets, epsilon-productions
    first = {i: set() for i in grammar.nonterminals}
    first.update((i, {i}) for i in grammar.terminals)
    follow = {i: set() for i in grammar.nonterminals}
    epsilon = set()

    while True:
        updated = False

        for nt, expression in grammar.rules:
            # FIRST set w.r.t epsilon-productions
            for symbol in expression:
                updated |= union(first[nt], first[symbol])
                if symbol not in epsilon:
                    break
            else:
                updated |= union(epsilon, {nt})

            # FOLLOW set w.r.t epsilon-productions
            aux = follow[nt]
            for symbol in reversed(expression):
                if symbol in follow:
                    updated |= union(follow[symbol], aux)
                if symbol in epsilon:
                    aux = aux.union(first[symbol])
                else:
                    aux = first[symbol]

        if not updated:
            return first, follow, epsilon


CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.58 µs


In [3]:
%%time
def union(first, begins):
    n = len(first)
    first |= begins
    return len(first) != n

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 9.06 µs


In [4]:
%%time
class Grammar:

    def __init__(self, *rules):
        self.rules = tuple(self._parse(rule) for rule in rules)

    def _parse(self, rule):
        return tuple(rule.replace(' ', '').split('::='))

    def __getitem__(self, nonterminal):
        yield from [rule for rule in self.rules if rule[0] == nonterminal]

    @staticmethod
    def is_nonterminal(symbol):
        return symbol.isalpha() and symbol.isupper()

    @property
    def nonterminals(self):
        return set(nt for nt, _ in self.rules)

    @property
    def terminals(self):
        return set(
            symbol
            for _, expression in self.rules
            for symbol in expression
            if not self.is_nonterminal(symbol)
        )

CPU times: user 43 µs, sys: 0 ns, total: 43 µs
Wall time: 47 µs


## left-recursive grammar w/ epsilon-production

In [5]:
%%time
first, follow, epsilon = first_and_follow(Grammar(
    '^ ::= A $',
    'A ::= ABBC',
    'A ::= B',
    'A ::= 1',
    'B ::= C',
    'B ::= 2',
    'C ::= 3',
    'C ::= ',
))

CPU times: user 140 µs, sys: 31 µs, total: 171 µs
Wall time: 175 µs


In [6]:
%%time
first

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs


{'C': {'3'},
 '^': {'$', '1', '2', '3'},
 'A': {'1', '2', '3'},
 'B': {'2', '3'},
 '3': {'3'},
 '$': {'$'},
 '2': {'2'},
 '1': {'1'}}

In [7]:
%%time
epsilon

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.77 µs


{'A', 'B', 'C'}

## arithmetic expressions

In [8]:
%%time
first, follow, epsilon = first_and_follow(Grammar(
    '^ ::= E $',
    'E ::= E + T',
    'E ::= T',
    'T ::= T * F',
    'T ::= F',
    'F ::= ( E )',
    'F ::= x',
))

CPU times: user 72 µs, sys: 15 µs, total: 87 µs
Wall time: 89.9 µs


In [9]:
%%time
first

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs


{'^': {'(', 'x'},
 'T': {'(', 'x'},
 'E': {'(', 'x'},
 'F': {'(', 'x'},
 '*': {'*'},
 '+': {'+'},
 '(': {'('},
 '$': {'$'},
 ')': {')'},
 'x': {'x'}}

In [10]:
%%time
follow

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


{'^': set(),
 'T': {'$', ')', '*', '+'},
 'E': {'$', ')', '+'},
 'F': {'$', ')', '*', '+'}}

In [11]:
%%time
epsilon

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


set()

# The End